<a href="https://colab.research.google.com/github/PedroFerreiradaCosta/NeuroadaptiveEEG/blob/main/FaceShiftBirkbeck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/PedroFerreiradaCosta/FaceFitOpt/master/Figures/facefitopt_logo.png" alt="FaceFitOpt Logo" width="1000">

In [ ]:
#@markdown #**Load pictures and generate images between the two faces**

# Clone my repo
!git clone https://github.com/PedroFerreiradaCosta/stylegan-encoder.git
import os
os.chdir("stylegan-encoder")

# Import dependencies
%tensorflow_version 1.x
import pickle
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
from google.colab import files

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import tensorflow as tf
# path to model code and weight
path_model = 'https://drive.google.com/uc?id=1I1vyhKJhoQul3ryIrHcTlpYycPAhWooI'


""" create tf session """
yn_CPU_only = False

if yn_CPU_only:
    config = tf.ConfigProto(device_count = {'GPU': 0}, allow_soft_placement=True)
else:
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True

sess = tf.InteractiveSession(config=config)

try:
    with dnnlib.util.open_url(path_model, 'rb') as file:
        G, D, Gs = pickle.load(file)
except FileNotFoundError:
    print('before running the code, download pre-trained model to project_root/asset_model/')
    raise


try:
  generator = Generator(Gs, batch_size=1, randomize_noise=False)
except ValueError:
  print("Restarting session. Run code again")
  os.kill(os.getpid(), 9)

  # Pick latent vector.
rnd = np.random.RandomState(5)
latents = rnd.randn(1, Gs.input_shape[1])

# Generate image.
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)
images = tflib.convert_images_to_uint8(images)
  
def generate_image(latent_vector, i):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    img.save("tmp{:02d}.png".format(i+1))
    return img.resize((256, 256))


# Create folders to align and generate artificial image
image_path = Path.cwd()
Path.mkdir(image_path / 'aligned_images', exist_ok=True)
Path.mkdir(image_path / 'generated_images', exist_ok=True)
Path.mkdir(image_path / 'raw_images', exist_ok=True)

try:
  path_to_remove = Path(image_path / 'raw_images' / '.ipynb_checkpoints')
  path_to_remove.rmdir()
except FileNotFoundError:
  print('No such file exists.')

print()
print()

def upload_files():
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(f'raw_images/{k}', 'wb').write(v)
  return list(uploaded.keys())

if os.listdir('raw_images/')==[]:
  uploaded_images = upload_files()
else:
  for i in os.listdir('raw_images/'):
    os.remove(i)
  uploaded_images = upload_files()
# 1) Extract and align faces from images
# Images to be processed should be added to the folder 'raw_images'
!python align_images.py raw_images/ aligned_images/
# if downloading dlib face landmarks takes more than 1/2 min, rerun code

# 2) Find latent representation of aligned images
!python encode_images.py aligned_images/ generated_images/ latent_representations/ --iterations 500


# src_dir - directory with images
# generaed_miages_dir - disrectory for storing generated images
# dlatent_dir - directory for storing dlatent representations
# batch_size - df - 1 - for generator and perceptual model
# image_size - df - 256 - Size of images for perceptual model
# lr - df 1 - learning rate for perc. model
# iterations - df - 1000 - opetimizations steps per batch 
# randomize_noise-  df - False - add noise to dlatents during optimization

# Load results
mom = np.load(f'latent_representations/{uploaded_images[0][:-4]}_01.npy')
notmom = np.load(f'latent_representations/{uploaded_images[1][:-4]}_01.npy')

# Print and download (10 a.t.m.)
alpha = [1,0.88,0.77,0.66,0.55,0.44,0.33,0.22,0.11, 0]
for i in range(len(alpha)):
    plt.figure()
    new_latent_vector = (alpha[i]*mom)+((1-(alpha[i]))*notmom)
    plt.imshow(generate_image(new_latent_vector, i))
    plt.axis('off')
    plt.show()
    files.download("tmp{:02d}.png".format(i+1)) 

# download originals
os.rename(f'aligned_images/{uploaded_images[0][:-4]}_01.png', 'tmp00.png')
os.rename(f'aligned_images/{uploaded_images[1][:-4]}_01.png', 'tmp11.png')
files.download("tmp00.png")
files.download("tmp11.png")